In [25]:
from openai import OpenAI
import pandas as pd

In [27]:
# Parameters

few_shot_no = 25  # Number of examples given for few-shot learning
test_no = 500     # Number of testing examples

API_KEY = 'your_API_key_here'

In [28]:
# Pick examples for few-shot learning randomly from training set

train_path = "/home/gowri/GDrive/Projects/cryptics-llm/No_Commit/Data/data_cryptic_train.json"

df_train = pd.read_json(train_path, lines=True)
df_train = df_train.sample(n=few_shot_no, random_state=42)

In [29]:
# Pick testing examples from test set

test_path = "/home/gowri/GDrive/Projects/cryptics-llm/No_Commit/Data/data_cryptic_test.json"

df_test = pd.read_json(test_path, lines=True)
df_test = df_test.sample(n=test_no, random_state=42)

In [30]:
# Build context for few-shot learning

context=[{"role": "system", "content": "You are a cryptic crossword solver. Following the examples given to you, give the answer to the crossword clue."}]

for index, data in df_train.iterrows():
    datadict = {}
    datadict["role"] = "user"
    datadict["content"] = "Solve the following cryptic crossword clue: " + data["clue"]
    context.append(datadict)
    
    datadict = {}
    datadict["role"] = "assistant"
    datadict["content"] = data["ans"]
    context.append(datadict)   

In [31]:
# Prompt ChatGPT to answer the crossword clues and save the predictions

client = OpenAI(api_key=API_KEY)
pred_list = []

for i in range(test_no):
    question = [{"role": "user", "content": "Solve the following cryptic crossword clue: " + df_test.iloc[i]["clue"]}]

    completion = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages= context+question
    )

    pred_list.append(completion.choices[0].message.content)

In [34]:
# Print accuracy

df_test["pred"] = pred_list
count = df_test[df_test['ans']==df_test['pred']].shape[0]
print("Number of right answers = ", count)
print("Percent ccuracy on cryptics test dataset = ", 100 * count/test_no)

Number of right answers =  50
Percent ccuracy on cryptics test dataset =  10.0
